<a href="https://colab.research.google.com/github/viercame/webscrapping/blob/main/Merqueo_webscrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium



In [ ]:
import sys
import csv
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import asyncio
from bs4 import BeautifulSoup
import json
import re

from google.colab import drive
drive.mount('drive')

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time


#options = Options()
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
driver = webdriver.Chrome('chromedriver', options=options)
driver.maximize_window()

#paginas a explorar
paginas=[
         'marcas-exclusivas',
         'lacteos-y-huevos',
         'abarrotes',
         'ahorra-en-grande',
         'despensa',
         'pollo-carne-y-pescado',
         'carnes-frias-y-embutidos',
         'congelados',
         'cuidado-de-la-ropa',
         'aseo-del-hogar',
         'cuidado-personal',
         'bebes',
         'super-ofertas',
         'panaderia-y-arepas',
         'cafe-chocolate-y-te',
         'galletas-dulces-y-antojos',
         'pasabocas-mecato',
         'saludables',
         'bebidas',
         'cervezas',
         'licores',
         'vinos',
         'hogar',
         'mascotas',
         'drogueria'
]



#Define parámetros a tomar desde la página
dict_param=[
            'data-name',
            'data-price',
            'data-special'
]
dfmerqueo=pd.DataFrame()

fecha = '2021-10-27'

for x in paginas:
    print (x)
    url='https://merqueo.com/bogota/super-ahorro/' + x

    driver.get(url)


    #################### RUTINA PARA ACTIVAR TODOS LOS PRODUCTOS CUANDO LA PAGINA REQUIERE BAJAR HASTA EL FINAL PARA MOSTRARLOS ######################################

    pre_scroll_height = driver.execute_script('return document.body.scrollHeight;')
    #print(pre_scroll_height)
    run_time, max_run_time = 0, 3

    while True:
        iteration_start = time.time()
        # Scroll webpage, the 100 allows for a more 'aggressive' scroll
        try:
            driver.execute_script('window.scrollTo(0, 100*document.body.scrollHeight);')
            post_scroll_height = driver.execute_script('return document.body.scrollHeight;')
        except TimeoutException as ex:
            break

        #print(post_scroll_height)
        scrolled = post_scroll_height != pre_scroll_height
        #print(scrolled)
        timed_out = run_time >= max_run_time

        if scrolled:
            run_time = 0
            pre_scroll_height = post_scroll_height
        elif not scrolled and not timed_out:
            run_time += time.time() - iteration_start
        elif not scrolled and timed_out:
            break


    #time.sleep(2)

    soup=BeautifulSoup(driver.page_source, 'lxml')

    driver.quit

    product_info=soup.find_all('aside', class_="mq-product-actions btn-add")
    product_href = soup.find_all('h3', class_='mq-product-title')
    product_subtitle = soup.find_all('p', class_='mq-product-subtitle')
    temp={}
    count=0
    for element in product_info:
        temp['productName'] = element['data-name'] + " - " + product_subtitle[count].get_text()
        temp['productId'] = element['data-id']
        temp['highPrice'] = element['data-price']
        temp['categoria1'] = x
        try:
            temp['lowPrice'] = element['data-special']
        except:
            temp['lowPrice'] = element['data-price']
        temp['link']='https://merqueo.com' + product_href[count].find('a', href=True)['href']
        count = count +1
        dfmerqueo = dfmerqueo.append(temp, ignore_index=True) #Almacena en un dataframe
        
        dfmerqueo['fecha']=fecha
        dfmerqueo['almacen']='MERQUEO'

#Exporta a un csv
nombrearchivo = fecha + ' merqueo.csv'
dfmerqueo.to_csv("/content/drive/MyDrive/Vicente/Python/Mercado/" + nombrearchivo, index=False)


Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
marcas-exclusivas
lacteos-y-huevos
abarrotes
ahorra-en-grande
despensa
pollo-carne-y-pescado
carnes-frias-y-embutidos
congelados
cuidado-de-la-ropa
aseo-del-hogar
cuidado-personal
bebes
super-ofertas
panaderia-y-arepas
cafe-chocolate-y-te
galletas-dulces-y-antojos
pasabocas-mecato
saludables
bebidas
cervezas
licores
vinos
hogar
mascotas
drogueria


In [ ]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [ ]:
dfmerqueo

,categoria1,highPrice,link,lowPrice,productId,productName,fecha,almacen
0,marcas-exclusivas,3250,https://merqueo.com/bogota/super-ahorro/marcas...,2763,586068,Agua Uzu sin gas garrafa - \n 5000 Ml\n,2021-10-27,MERQUEO
1,marcas-exclusivas,3300,https://merqueo.com/bogota/super-ahorro/marcas...,2805,586067,Agua Uzu sin gas X6 - \n 300 Ml\n,2021-10-27,MERQUEO
2,marcas-exclusivas,7800,https://merqueo.com/bogota/super-ahorro/marcas...,6630,594727,Queso Tipo Sandwich Tajado Dimistri - \n ...,2021-10-27,MERQUEO
3,marcas-exclusivas,9390,https://merqueo.com/bogota/super-ahorro/marcas...,7982,594766,Queso Mozzarella Tajado Pure Valley - \n ...,2021-10-27,MERQUEO
4,marcas-exclusivas,10390,https://merqueo.com/bogota/super-ahorro/marcas...,8832,594770,Queso Mozzarella Bloque Pure Valley - \n ...,2021-10-27,MERQUEO
...,...,...,...,...,...,...,...,...
6316,drogueria,7200,https://merqueo.com/bogota/super-ahorro/drogue...,6120,1343,Tarrito Rojo Jgb Tradicional - \n 135 Gr\...,2021-10-27,MERQUEO
6317,drogueria,45899,https://merqueo.com/bogota/super-ahorro/drogue...,39014,540308,Oferta Suplemento Multivitaminico Shot B 30 Ca...,2021-10-27,MERQUEO
6318,drogueria,26491,https://merqueo.com/bogota/super-ahorro/drogue...,22517,540309,Suplemento Multivitaminico Shot B 30 Capsulas ...,2021-10-27,MERQUEO
6319,drogueria,7780,https://merqueo.com/bogota/super-ahorro/drogue...,6613,1345,Tarrito Rojo Jgb Vainilla - \n 135 Gr\n,2021-10-27,MERQUEO


In [ ]:
dfmerqueo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6073 entries, 0 to 6072
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   categoria1   6073 non-null   object
 1   highPrice    6073 non-null   object
 2   link         6073 non-null   object
 3   lowPrice     6073 non-null   object
 4   productId    6073 non-null   object
 5   productName  6073 non-null   object
 6   fecha        6073 non-null   object
 7   almacen      6073 non-null   object
dtypes: object(8)
memory usage: 379.7+ KB
